In [38]:
from bs4 import BeautifulSoup
import csv
import logging
import pandas as pd
import requests
import time
import nltk
import spacy
import numpy as np
from nltk.text import Text

####
import warnings
warnings.filterwarnings('ignore')

# display max column width for descriptions
pd.set_option('display.max_colwidth', None)

In [39]:
# download and install necessary nltk / spacy packages
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

!pip install spacy
import spacy

!python3 -m spacy download en_core_web_sm

# load sm language model
nlp = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/danielevans/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/danielevans/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 7.4 MB/s eta 0:00:00m eta 0:00:010:00:01


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [40]:
# read in raw data and print out


df = pd.read_csv("raw.csv")  
df.head()

created_published  date  \
0               ['Abbeville, S.C. : Charles H. Allen, 1847-1869.']  1847   
1                                ['Abbeville, S.C. : Hugh Wilson']  1865   
2                          ['Abbeville, S.C. : Bonham and Perrin']  1884   
3         ['Abbeville, S.C. : W.A. Lee & Hugh Wilson, 1869-1924.']  1869   
4  ['Abbeville, S.C. : W.A. Lee and Hugh Wilson, Jr., 1860-1869.']  1860   

  dates_of_publication  \
0            1847-1869   
1            1865-1865   
2            1884-1887   
3            1869-1924   
4            1860-1869   

                                                                                                                                                                                                                                                                                                                                                                                                                                                description  \
0                                                                                                              ['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, no. 47 (Sept. 29, 1869). Publication suspended, <1861-1865>. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville press (DLC)sn 85042527 (OCoLC)12622573 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']   
1                                                                                                         ['Weekly Began in July 1865. Ceased with Aug. 31, 1865. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Published during the suspension of the Abbeville press. Description based on: Vol. 1, no. 3 (Aug. 10, 1865). Latest issue consulted: Vol. 1, no. 6 (Aug. 31, 1865).']   
2                                                                                                                                  ['Weekly Began Oct. 1, 1884; ceased in 1887. Cf. Watchman and southron, July 20, 1887. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Description based on: Vol. 1, no. 1 (Oct. 1, 1884). Latest issue consulted: Vol. 3, no. 38 (June 28, 1887).']   
3  ['Triweekly, Jan. 7, 1920-Feb. 13, 1924 Began with Oct. 1, 1869; ceased with Feb. 13, 1924. Title varies slightly. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Formed by the union of: Abbeville press and: Abbeville banner. Description based on: Vol. 17, no. 23 (Oct. 1, 1869). Latest issue consulted: Feb. 13, 1924. Press and banner (DLC)  2013228796 (OCoLC)841312126']   
4                                                                                  ['Weekly Vol. 8, no. 28 (Nov. 9, 1860)-v. 17, no. 22 (Sept. 24, 1869). Suspended <1864>-Aug. 1865. Resumed with Sept. 7, 1865 issue. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville banner (DLC)sn 85026945 (OCoLC)12795764 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

In [41]:
# remove html tags. no worries about error

def stripped_strings(text):
    try:
        return (' '.join(BeautifulSoup(text).stripped_strings))
    except TypeError as e:
        print(e)


df['essay'] = [stripped_strings(text) for text in df['essay'] ]
    

object of type 'float' has no len()


In [42]:
# print w. out html tags
df

,created_published,date,dates_of_publication,description,essay,essay_contributor,language,latlong,location,raw_lccn,subjects,title,url
0,"['Abbeville, S.C. : Charles H. Allen, 1847-1869.']",1847,1847-1869,"['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, no. 47 (Sept. 29, 1869). Publication suspended, <1861-1865>. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville press (DLC)sn 85042527 (OCoLC)12622573 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']","For nearly a century, the Abbeville Press and Banner (1844-1925) reported on international, national, and local news for the citizens of Abbeville County, located in the upstate region of South Carolina. The N.W. Ayer & Son’s American Newspaper Annual described Abbeville County simply as a “cotton growing district,” which while true on the surface is also slightly misleading. Although the cultivation of cotton had indeed dominated the county’s economy for most of the 19 th century, in the 1890s Abbeville was undergoing a period of economic growth, starting with the arrival of the Georgia, Carolina, and Northern Railroad (later called the Seaboard Air Line) in 1892 and the founding of the Abbeville Cotton Mill Company in 1895. Other significant developments in the region included: the establishment of an African American school, called Ferguson Academy (later renamed Harbison Agricultural College), in 1868; the founding of Calhoun Falls at the junction of the Seaboard Air Line and Savannah Valley Railroad in the early 1890s; and the creation of Greenwood and McCormick Counties from portions of Abbeville County in 1899 and 1916. The Abbeville Press and Banner began as two weekly newspapers, the Abbeville Banner and Abbeville Press . Charles Henry Allen and Henry S. Kerr had established the Abbeville Banner in 1844. Its competitor, the Abbeville Independent Press (later known as the Abbeville Press ) began in 1853. Coleman C. Puckett and George W. Fant brought considerable experience to their jobs as publishers of the Abbeville Independent Press , Puckett having previously edited the Anderson Southern Rights Advocate and Fant having apprenticed with the Anderson Gazette . During the Civil War, the Abbeville Banner suspended publication. Hugh Wilson, Jr. and William Augustus Lee, the proprietors of the Abbeville Press , turned over publishing duties to a former business partner and enlisted in the Confederate States Army. Sadly, few issues of either newspaper have survived from the 1860s. After the war, Wilson and Lee bought the Abbeville Banner and merged it with the Abbeville Press to create the Abbeville Press and Banner . The first issue appeared on October 1, 1869. Like other white Democrat newspapers of the era, the Press and Banner steered a conservative course, celebrating the return of the “Bourbons,” or antebellum-era aristocrats, to political power in 1877 and championing the interests of agrarian elites. Throughout its existence, it faced only one true competitor, the Abbeville Medium , which existed from 1871 through 1923. Circulation rates for the Press and Banner generally hovered between 800 and 1,200 subscriptions. Several of the men who ran these newspapers were leaders in their communities. George W. Fant, who was briefly involved with the Independent Press , served as postmaster and director of the National Bank of Anderson. Hugh Wilson, Jr., in addition to editing and publishing the Press and Banner , served as director of and stockholder in the Abbeville Cotton Mill and was also an officer in the South Carolina Press Association. Fellow proprietor William Augustus Lee practiced real estate law, represented the Abbeville District in the South Carolina Constitutional Convention of 1865, and served as a state representative in 1865-66. Starting in the 1920s, the Abbeville Press and Banner underwent a series of changes, from adopting a triweekly publication schedule to shortening it

In [43]:
# convert to string, not sure why a certain number of essays are being fed in as a float.
df["essay"] = df["essay"].astype(str)

In [44]:
# create unigrams from essays
start = time.time()
df["unigrams"] = df["essay"].apply(nltk.word_tokenize)
print(time.time() - start)

4.2335193157196045


In [45]:
# print out unigrams
# give context for all of these things
# terms that are unique for this process 
# explain terms
text = df['unigrams'].apply(Text)
text.head()

0                                           (For, nearly, a, century, ,, the, Abbeville, Press, and, Banner, (, 1844-1925, ), reported, on, international, ,, national, ,, and, local, news, for, the, citizens, of, Abbeville, County, ,, located, in, the, upstate, region, of, South, Carolina, ., The, N.W, ., Ayer, &, Son, ’, s, American, Newspaper, Annual, described, Abbeville, County, simply, as, a, “, cotton, growing, district, ,, ”, which, while, true, on, the, surface, is, also, slightly, misleading, ., Although, the, cultivation, of, cotton, had, indeed, dominated, the, county, ’, s, economy, for, most, of, the, 19, th, century, ,, in, the, 1890s, Abbeville, was, undergoing, a, ...)
1            (The, short-lived, weekly, Abbeville, Bulletin, (, July-August, 1865, ), offers, a, glimpse, into, a, community, adapting, to, new, realities, under, challenging, circumstances, ., In, the, early, 1860s, ,, the, white, citizens, of, Abbeville, County, (, then, Abbeville, District, ), ,, South

In [46]:
# tokenize sentences, apply pos tagging. not necessary w spacy but good to include

token_sent = df.essay.apply(nltk.word_tokenize)  
tagged_sent= token_sent.apply(nltk.pos_tag)
tagged_sent

0                                                         [(For, IN), (nearly, RB), (a, DT), (century, NN), (,, ,), (the, DT), (Abbeville, NNP), (Press, NNP), (and, CC), (Banner, NNP), ((, (), (1844-1925, JJ), (), )), (reported, VBD), (on, IN), (international, JJ), (,, ,), (national, JJ), (,, ,), (and, CC), (local, JJ), (news, NN), (for, IN), (the, DT), (citizens, NNS), (of, IN), (Abbeville, NNP), (County, NNP), (,, ,), (located, VBN), (in, IN), (the, DT), (upstate, JJ), (region, NN), (of, IN), (South, NNP), (Carolina, NNP), (., .), (The, DT), (N.W, NNP), (., .), (Ayer, NNP), (&, CC), (Son, NNP), (’, NNP), (s, VBD), (American, JJ), (Newspaper, NNP), (Annual, NNP), (described, VBD), (Abbeville, NNP), (County, NNP), (simply, RB), (as, IN), (a, DT), (“, JJ), (cotton, NN), (growing, VBG), (district, NN), (,, ,), (”, NNP), (which, WDT), (while, IN), (true, JJ), (on, IN), (the, DT), (surface, NN), (is, VBZ), (also, RB), (slightly, RB), (misleading, VBG), (., .), (Although, IN), (the, DT), (c

In [47]:
token_sent

0                                                [For, nearly, a, century, ,, the, Abbeville, Press, and, Banner, (, 1844-1925, ), reported, on, international, ,, national, ,, and, local, news, for, the, citizens, of, Abbeville, County, ,, located, in, the, upstate, region, of, South, Carolina, ., The, N.W, ., Ayer, &, Son, ’, s, American, Newspaper, Annual, described, Abbeville, County, simply, as, a, “, cotton, growing, district, ,, ”, which, while, true, on, the, surface, is, also, slightly, misleading, ., Although, the, cultivation, of, cotton, had, indeed, dominated, the, county, ’, s, economy, for, most, of, the, 19, th, century, ,, in, the, 1890s, Abbeville, was, undergoing, a, ...]
1                 [The, short-lived, weekly, Abbeville, Bulletin, (, July-August, 1865, ), offers, a, glimpse, into, a, community, adapting, to, new, realities, under, challenging, circumstances, ., In, the, early, 1860s, ,, the, white, citizens, of, Abbeville, County, (, then, Abbeville, District, )

In [48]:
# count of words. e.g. 'century'
t = token_sent.apply(lambda x: nltk.Text(x).count('century'))
t

0       2
1       0
2       1
3       2
4       2
       ..
3675    0
3676    0
3677    0
3678    0
3679    0
Name: essay, Length: 3680, dtype: int64

In [50]:
# concordance search, grab 35 characters before / after word
# give context
t = token_sent.apply(lambda x: nltk.Text(x).concordance("printer"))

no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
 was elected to the Office of State Printer and moved to the state capitol , Sa
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
espectively . Hughes , trained as a printer , became co-owner of the Albuquerqu
no matches
Displaying 4 of 4 matches:
er achieving success as a newspaper printer in Baltimore , Matthew Brown ( 1774
 1774-1831 ) , Philadelphia-trained printer , hoped to launch a Federalist

Displaying 1 of 1 matches:
s both a grammar school teacher and printer . In March 1834 , he disseminated t
Displaying 1 of 1 matches:
s both a grammar school teacher and printer . In March 1834 , he disseminated t
Displaying 1 of 1 matches:
s both a grammar school teacher and printer . In March 1834 , he disseminated t
Displaying 1 of 1 matches:
s both a grammar school teacher and printer . In March 1834 , he disseminated t
Displaying 1 of 1 matches:
s both a grammar school teacher and printer . In March 1834 , he disseminated t
Displaying 1 of 1 matches:
s both a grammar school teacher and printer . In March 1834 , he disseminated t
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
t of his life . He apprenticed as a printer , served in the Civil War , and the
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matche

no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
1865 . Mighels was elected as state printer in 1868 and oversaw all government 
no matches
no matches
no matches
Displaying 1 of 1 matches:
ackseat as business manager and job printer . Due to ill health , Johnson retir
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
s considered an able journalist and printer , Rowlenson changed the name of the
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
itr

no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
newspaper to others , including his printer , Rollin C. Brown . After Anson Pac
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
 Marschalk , Jr. , son of the first printer in the state , Andrew Marschalk , S
no matches
no matches
no matches
Displaying 1 of 1 matches:
onsiderations . After three years , printer L.O . Littlefield left the paper an
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
no matches
Displaying 1 of 1 matches:
he paper in 1847 while working as a printer in the newspaper office . Dyke sold
no matches
no matches
no matches
Displaying 1 of 1 matches:
itrant subscribers “ believe that a printer lives on his imagination , his neig
no 

In [22]:
####

In [23]:
#df.head(2)

In [24]:
## Need to write this so each line goes to the df
## write this as a function
## re-write append into concat to deal w depreciation warnings
## 

%time

df['essay']= df['essay'].astype(str)
ppl_df = pd.DataFrame()


for row in range(len(df)):
    people = {}
    lccn = df.loc[row, "raw_lccn"]
    doc = nlp(df.loc[row, "essay"])
#     for np in doc.noun_chunks:
#         print(np.text)
    person = [ent.text for ent in doc.ents if ent.label_ == 'PERSON']
    orgs = [ent.text for ent in doc.ents if ent.label_ == 'ORG']
    lccn = str(lccn).strip()
    people[lccn] = [person, orgs]
    t = pd.DataFrame.from_dict(people, orient='index', columns=["people","organization"])
    ppl_df = ppl_df.append(t)

#     test['person'] = docs.ents.apply(ent.text if ent.label == 'PERSON')
#     orgs = [ent.text for ent in doc.ents if ent.label_ == 'ORG']
#     LOC = [ent.text for ent in doc.ents if ent.label_ == 'LOC']
#     person = pd.DataFrame(person)
#     print(person)
#     test['person'] = person.loc[row, "essay"]


CPU times: user 1e+03 ns, sys: 1e+03 ns, total: 2 µs
Wall time: 2.86 µs


In [25]:
ppl_df.tail(50)

,people,organization
sn 85033078,"[Whigs, Rapids, Centralia, John N. Brundage, James E. Ingraham, Ingraham, Philleo, Paul, Albert L. Fontaine, Paul, Albert]","[the Republican Party, Army, the Republican Party, Nekoosa, Christian Temperance Union]"
sn 86091172,"[Picotte, Broadford, T.E. Picotte, Marshall]","[the Wood River Times, Wood River Times, Picotte’s Times, the Democratic Wood River News, Hailey, Wood River Times, Hailey, Bellevue, Bullion, Times, Hailey, the anti-Chinese League, Times, Times]"
sn 90050109,"[William Edward Woodruff, Cyrus McCrory, Wade McCrory, Cotton, Wade McCrory, Gustave W. Kramer, Kramer, Walter Wilson, Raney, Kramer, Raney, Raney, Gustave Kramer, Ernest Carl Kramer, Ernest Kramer, Charles M. James, James, Raney, Kramer, Raney, Charles James, James]","[the Arkansas Gazette, the Bank of McCrory, the Woodruff County News, the Arkansas Supreme Court, the News Publishing Company, Kramer, Supreme Court, Raney, McCrory, the Woodruff County News]"
sn 84023295,"[Martin Van Buren, Van Buren, Charles G. Eastman, Eastman, Woodstock Age, Edgar A. Kimball, Kimball, Edward M. Brown, Brown, Montpelier, Charles Eastman’s, William D. McMaster, Edward C. Dana, Eastman, Kimball, Brown, McMaster, McMaster, Edward Dana]","[the Burlington Weekly Free Press, Democratic Party, Courier, the Vermont Patriot, Kimball, the Elm Tree Press]"
sn 87065674,"[Woodville, Jefferson Davis, Missourian William A. A. Chisholm, Woodville Republican, Woodville Republican, Woodville Republican, Wilkinson Advertiser, William H. Nobles, John South Lewis, Andrew Lewis, John S. Lewis’s, Theodore Roosevelt]","[the Old Natchez Cession, Rosemont Plantation, Chisholm, Wilkinson Weekly Advertiser, Wilkinson Weekly Advertiser, the Democratic Party, Confederate, the Nashville Convention, the “Official Journal of Wilkinson County]"
sn 84020021,"[Woodville, Jefferson Davis, Missourian William A. A. Chisholm, Woodville Republican, Woodville Republican, Woodville Republican, Wilkinson Advertiser, William H. Nobles, John South Lewis, Andrew Lewis, John S. Lewis’s, Theodore Roosevelt]","[the Old Natchez Cession, Rosemont Plantation, Chisholm, Wilkinson Weekly Advertiser, Wilkinson Weekly Advertiser, the Democratic Party, Confederate, the Nashville Convention, the “Official Journal of Wilkinson County]"
sn 84020023,"[Woodville, Jefferson Davis, Missourian William A. A. Chisholm, Woodville Republican, Woodville Republican, Woodville Republican, Wilkinson Advertiser, William H. Nobles, John South Lewis, Andrew Lewis, John S. Lewis’s, Theodore Roosevelt]","[the Old Natchez Cession, Rosemont Plantation, Chisholm, Wilkinson Weekly Advertiser, Wilkinson Weekly Advertiser, the Democratic Party, Confederate, the Nashville Convention, the “Official Journal of Wilkinson County]"
sn 87065677,"[Woodville, Jefferson Davis, Missourian William A. A. Chisholm, Woodville Republican, Woodville Republican, Woodville Republican, Wilkinson Advertiser, William H. Nobles, John South Lewis, Andrew Lewis, John S. Lewis’s, Theodore Roosevelt]","[the Old Natchez Cession, Rosemont Plantation, Chisholm, Wilkinson Weekly Advertiser, Wilkinson Weekly Advertiser, the Democratic Party, Confederate, the Nashville Convention, the “Official Journal of Wilkinson County]"
sn 89060127,"[Samuel M. Crockett, W.B. Vane, Samuel M. Crockett, Messenger, the Somerset Herald, Bay craft, Edd Watson, H.L. Mencken, Matthew Williams, Mencken, George Armwood]","[the Maryland, Newtown, Eastern Shore]"
sn 89077510,"[Andrew Cameron, Cameron]","[The Chicago Workingman’s Advocate, the Chicago Typographical Union, the Chicago Times, Times, the Chicago Tribune, Times, Northwest, Evergreen, Times, the Typographical Union, Times, Cameron, Times, the National Labor Union, NLU, NLU, cooperatives, NLU, the Industrial Congress of the United States, the Inland Printer]"


In [ ]:
ppl_df.tail(50)

In [35]:
df_cd = pd.merge(df, ppl_df, how='inner', left_on = 'raw_lccn', right_index=True)

AttributeError: 'list' object has no attribute 'to_frame'

In [27]:
result.head()

,created_published,date,dates_of_publication,description,essay,essay_contributor,language,latlong,location,raw_lccn,subjects,title,url,unigrams,people,organization
0,"['Abbeville, S.C. : Charles H. Allen, 1847-1869.']",1847,1847-1869,"['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, no. 47 (Sept. 29, 1869). Publication suspended, <1861-1865>. Archived issues are available in digital format from the Library of Congress Chronicling America online collection. Abbeville press (DLC)sn 85042527 (OCoLC)12622573 Abbeville press and banner (DLC)sn 84026853 (OCoLC)10708843']","For nearly a century, the Abbeville Press and Banner (1844-1925) reported on international, national, and local news for the citizens of Abbeville County, located in the upstate region of South Carolina. The N.W. Ayer & Son’s American Newspaper Annual described Abbeville County simply as a “cotton growing district,” which while true on the surface is also slightly misleading. Although the cultivation of cotton had indeed dominated the county’s economy for most of the 19 th century, in the 1890s Abbeville was undergoing a period of economic growth, starting with the arrival of the Georgia, Carolina, and Northern Railroad (later called the Seaboard Air Line) in 1892 and the founding of the Abbeville Cotton Mill Company in 1895. Other significant developments in the region included: the establishment of an African American school, called Ferguson Academy (later renamed Harbison Agricultural College), in 1868; the founding of Calhoun Falls at the junction of the Seaboard Air Line and Savannah Valley Railroad in the early 1890s; and the creation of Greenwood and McCormick Counties from portions of Abbeville County in 1899 and 1916. The Abbeville Press and Banner began as two weekly newspapers, the Abbeville Banner and Abbeville Press . Charles Henry Allen and Henry S. Kerr had established the Abbeville Banner in 1844. Its competitor, the Abbeville Independent Press (later known as the Abbeville Press ) began in 1853. Coleman C. Puckett and George W. Fant brought considerable experience to their jobs as publishers of the Abbeville Independent Press , Puckett having previously edited the Anderson Southern Rights Advocate and Fant having apprenticed with the Anderson Gazette . During the Civil War, the Abbeville Banner suspended publication. Hugh Wilson, Jr. and William Augustus Lee, the proprietors of the Abbeville Press , turned over publishing duties to a former business partner and enlisted in the Confederate States Army. Sadly, few issues of either newspaper have survived from the 1860s. After the war, Wilson and Lee bought the Abbeville Banner and merged it with the Abbeville Press to create the Abbeville Press and Banner . The first issue appeared on October 1, 1869. Like other white Democrat newspapers of the era, the Press and Banner steered a conservative course, celebrating the return of the “Bourbons,” or antebellum-era aristocrats, to political power in 1877 and championing the interests of agrarian elites. Throughout its existence, it faced only one true competitor, the Abbeville Medium , which existed from 1871 through 1923. Circulation rates for the Press and Banner generally hovered between 800 and 1,200 subscriptions. Several of the men who ran these newspapers were leaders in their communities. George W. Fant, who was briefly involved with the Independent Press , served as postmaster and director of the National Bank of Anderson. Hugh Wilson, Jr., in addition to editing and publishing the Press and Banner , served as director of and stockholder in the Abbeville Cotton Mill and was also an officer in the South Carolina Press Association. Fellow proprietor William Augustus Lee practiced real estate law, represented the Abbeville District in the South Carolina Constitutional Convention of 1865, and served as a state representative in 1865-66. Starting in the 1920s, the Abbeville Press and Banner underwent a series of changes, from adopting a triweekly publicat

In [ ]:
ppl_df.head(3)

In [454]:
people

{'sn 85026945 ': ['Abbeville',
  'Calhoun Falls',
  'Greenwood',
  'Banner',
  'Charles Henry Allen',
  'Henry S. Kerr',
  'Coleman C. Puckett',
  'George W. Fant',
  'Puckett',
  'Hugh Wilson',
  'Jr.',
  'William Augustus Lee',
  'Lee',
  'George W. Fant',
  'Hugh Wilson',
  'Banner',
  'William Augustus Lee',
  'Fred Devereaux West'],
 'sn 93067670 ': ['Hugh Wilson',
  'Abbeville Banner',
  'Greenville Patriot',
  'James L. Orr',
  'Hugh Wilson',
  'William Augustus Lee',
  'Lee'],
 'sn 93067668 ': ['Abbeville Messenger',
  'Banner',
  'Abbeville Messenger',
  'Saluda Argus',
  'Saluda Argus',
  'Milledge Lipscomb Bonham',
  'James Sumter',
  'Thomas Perrin Cothran',
  'Saluda Argus',
  'Abbeville Messenger',
  'Abbeville Messenger',
  'Anderson Intelligencer',
  'Bonham',
  'Watchman',
  'Abbeville Messenger',
  'Abbeville Messenger',
  'Bonham',
  'Abbeville Messenger'],
 'sn 84026853 ': ['Abbeville',
  'Calhoun Falls',
  'Greenwood',
  'Banner',
  'Charles Henry Allen',
  'Henry 

In [442]:
test

,created_published,date,dates_of_publication,description,essay,essay_contributor,language,latlong,location,raw_lccn,subjects,title,url,unigrams
0,"['Abbeville, S.C. : Charles H. Allen, 1847-186...",1847,1847-1869,"['Weekly Vol. 4, no. 1 (Mar. 3, 1847)-v. 25, n...","For nearly a century, the Abbeville Press and ...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 85026945,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",The Abbeville banner.,https://www.loc.gov/item/sn85026945/,"[For, nearly, a, century, ,, the, Abbeville, P..."
1,"['Abbeville, S.C. : Hugh Wilson']",1865,1865-1865,['Weekly Began in July 1865. Ceased with Aug. ...,The short-lived weekly Abbeville Bulletin (Jul...,"['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'abbeville county', 'abbevil...",sn 93067670,"['Abbeville County (S.C.)--Newspapers', 'South...",The Abbeville bulletin.,https://www.loc.gov/item/sn93067670/,"[The, short-lived, weekly, Abbeville, Bulletin..."
2,"['Abbeville, S.C. : Bonham and Perrin']",1884,1884-1887,"['Weekly Began Oct. 1, 1884; ceased in 1887. C...",The Abbeville Messenger (1884-87) was one of s...,"['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'abbeville county', 'abbevil...",sn 93067668,"['Abbeville County (S.C.)--Newspapers', 'South...",The Abbeville messenger.,https://www.loc.gov/item/sn93067668/,"[The, Abbeville, Messenger, (, 1884-87, ), was..."
3,"['Abbeville, S.C. : W.A. Lee & Hugh Wilson, 18...",1869,1869-1924,"['Triweekly, Jan. 7, 1920-Feb. 13, 1924 Began ...","For nearly a century, the Abbeville Press and ...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 84026853,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",The Abbeville press and banner.,https://www.loc.gov/item/sn84026853/,"[For, nearly, a, century, ,, the, Abbeville, P..."
4,"['Abbeville, S.C. : W.A. Lee and Hugh Wilson, ...",1860,1860-1869,"['Weekly Vol. 8, no. 28 (Nov. 9, 1860)-v. 17, ...","For nearly a century, the Abbeville Press and ...","['University of South Carolina; Columbia, SC']",['english'],"[34.17895, -82.38025]","['united states', 'south carolina', 'abbeville...",sn 85042527,"['Abbeville (S.C.)--Newspapers', 'Abbeville Co...",Abbeville press.,https://www.loc.gov/item/sn85042527/,"[For, nearly, a, century, ,, the, Abbeville, P..."
5,"[""Abbeville, Vermilion Parish, La. : J.W. O'Br...",1913,1913-1944,"['Weekly Began with vol. 1, no. 1 (Mar. 1, 191...","Abbeville, the seat of Vermilion Parish, Louis...","['Louisiana State University; Baton Rouge, LA']",['english'],"[29.97333, -92.13575]","['united states', 'abbeville', 'louisiana', 'v...",sn 88064057,"['Abbeville (La.)--Newspapers', 'Vermilion Par...",Abbeville progress.,https://www.loc.gov/item/sn88064057/,"[Abbeville, ,, the, seat, of, Vermilion, Paris..."
6,"['Chicago, Ill. : Illinois Staats-Zeitungs-Com...",1894,1894-1899,"['Daily (except Sunday) 47. Jahrg., no. 253 (2...",The Illinois Staats-Zeitung (ISZ) was a German...,['University of Illinois at Urbana-Champaign L...,['german'],"[41.88425, -87.63245]","['cook', 'united states', 'illinois', 'chicago']",sn 83045004,['German Americans--Illinois--Chicago--Newspap...,Abendblatt,https://www.loc.gov/item/sn83045004/,"[The, Illinois, Staats-Zeitung, (, ISZ, ), was..."
7,"['Chicago, Ill. : [publisher not identified], ...",1891,1891-1894,['Daily (except Sunday) Began in 1891. -47. Ja...,The Illinois Staats-Zeitung (ISZ) was a German...,['University of Illinois at Urbana-Champaign L...,['german'],"[41.88425, -87.63245]","['cook', 'united states', 'illinois', 'chicago']",sn 83045003,['German Americans--Illinois--Chicago--Newspap...,Abendblatt der Illinois Staats-zeitung,https://www.loc.gov/item/sn83045003/,"[The, Illinois, Staats-Zeitung, (, ISZ, ), was..."
8,"[

# todo:
0. write this right.
1. check for and delete csv if does exist
2. (done) refactor / rate limit api calls.
3. (done) write to list or pickle before writing to csv
4. (done) test which sleep function is necessary
5. document code
6. NER / BERT
7. Begin planning for fall / next steps
8. Ingest / Clean xlsx
9. binderhub
10. send to M 9/2